In [3]:
!pip install nltk 
!pip install tensorflow
!pip install wordcloud
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from wordcloud import WordCloud
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

In [7]:
# Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Set plotting style
sns.set_style('whitegrid')
plt.style.use('fivethirtyeight')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lap\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lap\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\lap\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [17]:
import pandas as pd
import string
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# 1. Load dataset with encoding and error handling
try:
    df = pd.read_csv('C:/Users/lap/Downloads/data_news - data_news.csv', encoding='utf-8')
    print("\nDataset loaded successfully!")
except Exception as e:
    print(f"Error loading CSV: {e}")
    exit()

# 2. Clean and standardize column names
df.columns = df.columns.str.strip().str.lower()
print("\nCleaned Column Names:", df.columns)

# 3. Check if category column exists, rename if necessary
if 'category' not in df.columns:
    if 'Category' in df.columns:
        df.rename(columns={'Category': 'category'}, inplace=True)
    elif 'class' in df.columns:  # If class is the label
        df.rename(columns={'class': 'category'}, inplace=True)
    else:
        print("\n'category' column not found. Using available columns instead.")
        print("Available columns:", df.columns)

# 4. Ensure dataframe is not empty
if df.empty:
    print("DataFrame is empty. Check CSV loading or preprocessing steps.")
    exit()

# 5. Ensure text column exists
if 'text' not in df.columns:
    print("\nUsing the first column instead of 'text'")
    df.rename(columns={df.columns[0]: 'text'}, inplace=True)

# 6. Add text length column
df['text_length'] = df['text'].apply(len)

# 7. Plot with correct column name
if 'category' in df.columns:
    x_col = 'category'
else:
    x_col = df.columns[0]  # Fallback to first column

# Seaborn plot with corrected column reference
plt.figure(figsize=(10, 6))
sns.boxplot(x=x_col, y='text_length', data=df, palette='viridis')
plt.title('Text Length by Category')
plt.xlabel('Category')
plt.ylabel('Text Length (characters)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('text_length_by_category.png')
plt.close()

# 8. Preprocessing function
def preprocess_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    clean_text = ' '.join(tokens)
    return clean_text

# 9. Apply preprocessing
df['clean_text'] = df['text'].apply(preprocess_text)

# 10. Display cleaned samples
print("\nSample of Cleaned Text:")
for i in range(min(5, len(df))):
    print(f"Original: {df['text'][i]}")
    print(f"Cleaned: {df['clean_text'][i]}")
    print("")



Dataset loaded successfully!

Cleaned Column Names: Index(['category', 'headline', 'links', 'short_description', 'keywords'], dtype='object')

Using the first column instead of 'text'

Sample of Cleaned Text:
Original: WELLNESS
Cleaned: wellness

Original: WELLNESS
Cleaned: wellness

Original: WELLNESS
Cleaned: wellness

Original: WELLNESS
Cleaned: wellness

Original: WELLNESS
Cleaned: wellness



In [19]:
# Drop unnecessary columns
df = df.drop('links', axis=1)


In [21]:
# Fill missing values in 'keywords' with an empty string
df['keywords'].fillna('', inplace=True)


In [23]:
# Combine text columns
df['text'] = df[['headline', 'short_description', 'keywords']].agg(' '.join, axis=1)

# Drop the original columns
df = df.drop(['headline', 'short_description', 'keywords'], axis=1)


In [25]:
# Basic text cleaning
import re

def clean_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    stopwords = {'the', 'is', 'in', 'and', 'to', 'of', 'a', 'for', 'on', 'with', 'as', 'this', 'that'}
    tokens = [word for word in text.split() if word not in stopwords]
    return ' '.join(tokens)

# Apply cleaning function
df['cleaned_text'] = df['text'].map(clean_text)


In [29]:
# Check for column existence and rename if necessary
if 'category' not in df.columns:
    print("Column 'category' not found. Available columns:", df.columns)
else:
    # Feature extraction using TF-IDF
    from sklearn.feature_extraction.text import TfidfVectorizer

    tfidf = TfidfVectorizer(max_features=5000)
    X = tfidf.fit_transform(df['cleaned_text'].values)
    y = df['category'].values


Column 'category' not found. Available columns: Index(['text', 'text_length', 'clean_text', 'cleaned_text'], dtype='object')


In [35]:
# Identify the potential target column by checking for low cardinality
target_col = None

for col in df.columns:
    if df[col].nunique() < 20:  # Assuming the target column has fewer categories
        target_col = col
        break

if target_col:
    print(f"Using '{target_col}' as the target column.")
    y = df[target_col].values
else:
    raise KeyError("No suitable target column found in the dataset.")

# Split the data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


Using 'text_length' as the target column.


In [37]:
# Sampling 20% of the training data
X_train_sample, y_train_sample = X_train, y_train

# Further split to get 20% sample
X_train_sample, _, y_train_sample, _ = train_test_split(
    X_train_sample, y_train_sample, 
    test_size=0.8, 
    random_state=42, 
    stratify=y_train_sample
)


In [39]:
from sklearn.naive_bayes import MultinomialNB

# Initialize and fit the Naive Bayes model
nb_model = MultinomialNB()
nb_model.fit(X_train_sample, y_train_sample)


MultinomialNB()

In [41]:
# Predictions
y_pred = nb_model.predict(X_test)


In [43]:
from sklearn.metrics import accuracy_score, classification_report

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

# Display the results
print(f"Naïve Bayes Accuracy: {accuracy:.4f}")
print("\nClassification Report:\n")
print(report)


Naïve Bayes Accuracy: 0.6702

Classification Report:

              precision    recall  f1-score   support

           6       0.72      0.74      0.73      2000
           8       0.54      0.95      0.69      3000
           9       0.84      0.32      0.47      1000
          10       0.90      0.44      0.59      1000
          12       0.89      0.55      0.68      1000
          13       0.86      0.38      0.52      1000
          14       0.89      0.67      0.77      1000

    accuracy                           0.67     10000
   macro avg       0.81      0.58      0.64     10000
weighted avg       0.75      0.67      0.66     10000



Video Link:- https://drive.google.com/file/d/1J6jAsKW_V9ZyEYgfuO2CWKkEutucQ-cd/view?usp=sharing